In [ ]:
#!pip install networkx
#!pip install matplotlib

In [ ]:
from utils.problem import SearchProblem
from utils.graph_widget import show
from utils.graph import DirectedGraph, UndirectedGraph, Edge, Node


# Graph and Tree Search

Diese grundlegenden Implentierungen von Graph- und Baumsuche kann als Vorlage für verschiedenste Suchalgorithmen genutzt werden. In den meisten Fällen ist der einzige Unterschied zu anderen Suchverfahren wie Knoten der Fringe hinzu gefügt werden. Auch der Unterschied zwischen Graph und Baumsuche ist nicht sehr groß. Der einzige unterschied ist die *closed* Liste der Graphsuche. Diese Liste ist bei der Graphsuche nötig, weil allgemeine graphen cyclisch sein können, was zu unendlichen Suchzeiten führen kann wenn dies nicht berücksichtigt wird.

In [ ]:
# This differs between search algorithms
def expand_tree_node(node, fringe):
    fringe += node.children() 
    return fringe
        
def tree_search(problem):
    fringe=[problem.root]
    
    while True:
        if not fringe:  #check if fringe is empty
            return None
        
        node = fringe.pop(0)
        
        
        if node == problem.target_node:
            problem.backlog.add_log(node, fringe=fringe) # Ignore this
            return node
        
        
        fringe = expand_tree_node(node, fringe) 
        problem.backlog.add_log(node, fringe=fringe) # Ignore this
 


        
# This differs between search algorithms
def expand_graph_node(node, fringe, closed):
    for adj_node in node.neighbors():
        
        if adj_node not in closed and adj_node not in fringe:
            fringe.append(adj_node)
    return fringe


def graph_search(problem):
    closed = []
    fringe=[problem.start_node]
    while True:
        
        if not fringe: #check if fringe is empty
            return None
        
        
        node = fringe.pop(0)
        
        if node == problem.target_node:
            problem.backlog.add_log(node, fringe=fringe) # Ignore this
            return node
        
        if node not in closed:
            closed.append(node)
            fringe = expand_graph_node(node, fringe, closed)
        problem.backlog.add_log(node, fringe=fringe) # Ignore this

# Uninformed Search

Uninformierte Suche nutzt keine weiteren Informationen als die Verbindung zwischen Knoten. Daher gehen diese recht naiv vor, was aber häufig reicht.

## Breath First Search

Breitensuche (in diesem Fall als Baumsuche) hängt neue Knoten hinten an die Fringe an. Dadurch werden zuerst entdeckte Knoten zuerst behandelt.

In [ ]:
def expand_bf_tree_node(node, fringe):
    fringe += node.children() 
    return fringe
        
def breath_first_tree_search(problem):
    fringe=[problem.root]
    
    while True:
        if not fringe:  #check if fringe is empty
            return None
        
        node = fringe.pop(0)
        
        
        if node == problem.target_node:
            problem.backlog.add_log(node, fringe=fringe) # Ignore this
            return node
        
        
        fringe = expand_bf_tree_node(node, fringe) 
        problem.backlog.add_log(node, fringe=fringe) # Ignore this
        


## Depth First Search

Tiefensuche (in diesem Fall auch als Baumsuche) hängt neue Knoten vorne an die Fringe an. Dadurch werden die zuletzt entdeckten Knoten zuerst behandelt. Bei einem Baum kann man sich vorstellen, dass hier zuerst ein Ast bis zu den Blättern abgesucht wird wärend bei der Breitensuche die Äste gleichzeitig durchsucht werden. 

In [ ]:
def expand_df_tree_node(node, fringe):
    fringe = node.children() + fringe # This line makes all the difference
    return fringe
        
def depth_first_tree_search(problem):
    fringe=[problem.root]
    
    while True:
        if not fringe:  #check if fringe is empty
            return None
        
        node = fringe.pop(0)
        
        
        if node == problem.target_node:
            problem.backlog.add_log(node, fringe=fringe) # Ignore this
            return node
        
        
        fringe = expand_df_tree_node(node, fringe) 
        problem.backlog.add_log(node, fringe=fringe) # Ignore this
        


### Tree search Example

In [ ]:
nodes = [
    "A", 
    "B", 
    "C", 
    "D", 
    "E", 
    "F", 
    "G",  
    "H"
]

edges = [
    ("A", "B"),
    ("B", "C"),
    ("E", "A"),
    ("A", "G"),
    ("F", "D"),
    ("A", "F"),
    ("B", "H")

]

# Generate Graph
digraph = DirectedGraph(nodes, edges)

# Formulate search problem
search_problem = SearchProblem(start_node="E", target_node="C", graph=digraph)


# Execute search problem with a given algorithm
found_node = breath_first_tree_search(search_problem)
#found_node = depth_first_tree_search(search_problem)

# Visualise search
show(search_problem.backlog)

### Graph search Example

In [ ]:
nodes = [
    "A", 
    "B", 
    "C", 
    "D", 
    "E", 
    "F", 
    "G",  
    "H"
]

edges = [
    ("A", "B"),
    ("B", "C"),
    ("E", "A"),
    ("A", "G"),
    ("F", "D"),
    ("A", "F"),
    ("B", "H"),
    ("H", "F"),
    ("G", "D")

]

# Generate Graph
graph = UndirectedGraph(nodes, edges)

# Formulate search problem
undirected_problem = SearchProblem(start_node="E", target_node="C", graph=graph)

# Execute search problem with a given algorithm
found_node = graph_search(undirected_problem)

# Visualise search
show(undirected_problem.backlog)


# Informed Search
Informierte Suche nutzt im gegensatz zur uninformierten Suche weitere Informationen außer die Verbindung zwischen Knoten um Ziele effizienter zu finden. Sie folgt also der Idee "Best First". Ein bekanntes Beispiel dafür ist die A* Suche.

## Greedy best-first search
Hier wird der Eintrag als nächstes genommen der laut einer Heuristik *h* am besten ist. Bei Straßenverbindungen zwischen Städten kann man sich hier zum Beispiel die Luftlinie zwischen diesen als Heuristik vorstellen. 

In [ ]:
from collections import namedtuple
from heapq import heappush, heappop

FringeEntry = namedtuple('FringeEntry', 'f node previous_node')


def expand_greedy_best_first_node(node, fringe, closed):
    
    for adj_node in node.neighbors():
        
        if adj_node not in closed:
            f = adj_node.h
            heappush(fringe, FringeEntry(f, adj_node, node))
   
    return fringe
        

def greedy_best_first_search(problem):
    closed = []
    fringe=[]
    
    # Notice how we don't simply use a list anymore, but rather a heap queue. This data structure automatically 
    # sorts items by some priority. In the case of greedy best first search this priority is the value f (which
    # is equal to h in this search algorithm).
    heappush(fringe, FringeEntry(0, problem.start_node, None))
   
    while True:
        
       
        if not fringe: #check if fringe is empty
            return None
        
        
        # Again, we want to pop the best, not the first added. Since the heapq is already sorted to have the best
        # as first entry, we can simply pop now instead of searching for it.
        next_entry = heappop(fringe)
        node = next_entry.node
        

        
        if node == problem.target_node:
            problem.backlog.add_log(next_entry, fringe=fringe)
            return node
        
        if node not in closed:
            closed.append(node)
            fringe = expand_greedy_best_first_node(node, fringe, closed)
            
        problem.backlog.add_log(next_entry, fringe=fringe)

## A* Search
Die Idee ist möglichst immer den günstigsten weg als nächstes zu gehen. Zusätzlich zu der heuristic *h* die schon in der greedy best first search vorgestellt wurde, werden nun auch noch die bisherDie Kosten ergeben sich durch die Evaluierungsfunktion *f(n) = g(n) + h(n)*.

Wobei

*g(n)* = Bisherige Kosten um *n* zu erreichen

*h(n)* = Geschätze Kosten um von *n* zum Ziel zu kommen (Heuristik)

*f(n)* = Gesamte Geschätzen kosten vom start über *n* zum Ziel zu kommen.

In [ ]:
from collections import namedtuple
from heapq import heappush, heappop

AStarFringeEntry = namedtuple('FringeEntry', 'f g node previous_node')


def expand_A_star_node(node, current_g, fringe, closed):
    
    for adj_node in node.neighbors():
        
        if adj_node not in closed:
            g = current_g + node.cost_to[adj_node]
            f = g + adj_node.h
            heappush(fringe, AStarFringeEntry(f, g, adj_node, node))
   
    return fringe
        

def A_star_search(problem):
    closed = []
    fringe=[]
    
    # Notice how we don't simply use a list anymore, but rather a heap queue. This data structure automatically 
    # sorts items by some priority. In the case of A* search this priority is the value f.
    heappush(fringe, AStarFringeEntry(0, 0, problem.start_node, None))
   
    while True:
        
       
        if not fringe: #check if fringe is empty
            return None
        
        
        # Again, we want to pop the best, not the first added. Since the heapq is already sorted to have the best
        # as first entry, we can simply pop now instead of searching for it.
        next_entry = heappop(fringe)
        node = next_entry.node
        

        
        if node == problem.target_node:
            problem.backlog.add_log(next_entry, fringe=fringe)
            return node
        
        if node not in closed:
            closed.append(node)
            fringe = expand_A_star_node(node, next_entry.g, fringe, closed)
            
        problem.backlog.add_log(next_entry, fringe=fringe)

### Informed Search Example

In [ ]:
# Informed Search example


nodes = [
    Node("Hamburg"),
    Node("Göttingen"),
    Node("Berlin"),
    Node("Leipzig"),
    Node("Frankfurt"),
    Node("Nürnberg"),
    Node("Würzburg"),
    Node("Düsseldorf"),
    Node("Darmstadt"),
]

edges = [

    Edge("Hamburg", "Göttingen", cost=265),
    Edge("Berlin", "Hamburg", cost=284),
    Edge("Hamburg", "Düsseldorf", cost=404),
    Edge("Düsseldorf", "Frankfurt", cost=223),
    Edge("Göttingen","Frankfurt", cost=234),
    Edge("Würzburg", "Frankfurt", cost=122),
    Edge("Frankfurt", "Darmstadt", cost=42),
    Edge("Leipzig", "Göttingen", cost=228),
    Edge("Berlin", "Leipzig", cost=187),
    Edge("Leipzig", "Nürnberg", cost=282),
    Edge("Nürnberg", "Würzburg", cost=108),
    Edge("Würzburg", "Darmstadt", cost=132),
    Edge("Düsseldorf", "Darmstadt", cost=235),
    Edge("Göttingen", "Düsseldorf", cost=280),
]

# Luftlinie to Darmstadt
h = {
    "Hamburg": 419,
    "Frankfurt": 28,
    "Göttingen": 205,
    "Berlin": 443,
    "Leipzig": 309,
    "Nürnberg": 180,
    "Würzburg": 92,
    "Düsseldorf": 200,
    "Darmstadt": 0,
}

# Pick either directed or not directed
#graph = DirectedGraph(nodes, edges, heuristic=h)
graph = UndirectedGraph(nodes, edges, heuristic=h)

# Note: start can be changed, but target cannot, unless the heuristic h is adjusted accordingly
simple_problem = SearchProblem("Berlin", "Darmstadt", graph)

# Notice how greedy best first search finds the target fast, but doesn't find the quickest path

node = greedy_best_first_search(simple_problem)
#node = A_star_search(simple_problem)


show(simple_problem.backlog)